In [14]:
!pip install transformers datasets
!pip install torch  # for PyTorch backend


  Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)
  Using cached datasets-3.0.0-py3-none-any.whl (474 kB)
  Using cached regex-2024.9.11-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (782 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Using cached tqdm-4.66.5-py3-none-any.whl (78 kB)
  Using cached numpy-2.1.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.3 MB)
  Using cached packaging-24.1-py3-none-any.whl (53 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 KB 657.0 kB/s eta 0:00:00a 0:00:01
  Using cached PyYAML-6.0.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (751 kB)
  Using cached safetensors-0.4.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (435 kB)
  Using cached requests-2.32.3-py3-none-any.whl (64 kB)
  Using cached xxhash-3.5.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (194 kB)
  Using cached fsspec-2024.6.1-py3-none-a

In [15]:
!pip install tensorflow


  Using cached tensorflow-2.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (601.3 MB)
  Using cached h5py-3.11.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.3 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl (7.7 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl (133 kB)
  Using cached keras-3.5.0-py3-none-any.whl (1.1 MB)
  Using cached gast-0.6.0-py3-none-any.whl (21 kB)
  Using cached grpcio-1.66.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.7 MB)
  Using cached tensorboard-2.17.1-py3-none-any.whl (5.5 MB)
  Using cached ml_dtypes-0.4.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.2 MB)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl (24.5 MB)
  Using cached tensorflow_io_gcs_filesystem-0.37.1-cp310-cp310-manylinux_2

In [16]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch 

# Load the tokenizer and model for HeBERT
tokenizer = AutoTokenizer.from_pretrained("avichr/HeBERT")
model = AutoModelForMaskedLM.from_pretrained("avichr/HeBERT")

# Example usage: Tokenize some Hebrew text with a mask
text = "הספר הזה מאוד [MASK]"
inputs = tokenizer(text, return_tensors="pt")

# Run the model
with torch.no_grad():
    outputs = model(**inputs)

# Get predictions for the masked token
predictions = outputs.logits

# Decode the predictions
masked_index = torch.where(inputs.input_ids == tokenizer.mask_token_id)[1]
predicted_ids = torch.argmax(predictions[0, masked_index], dim=-1)
predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_ids)

print("Predicted tokens for masked position:", predicted_tokens)


/home/ameer/Kaleidoo/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Predicted tokens for masked position: ['מומלץ']


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Replace 'bert-base-uncased' with your HBERT model identifier
model_name = "bert-base-uncased"  # Example public model

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

print("Model and tokenizer loaded successfully.")


In [23]:
!pip install torchaudio


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
import torch
import torchaudio
from datasets import load_dataset, load_metric
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import re
test_dataset = load_dataset("common_voice", "he", split="test") # there is no common dataset for Hebrew, please, paste your data
wer = load_metric("wer")
processor = Wav2Vec2Processor.from_pretrained("imvladikon/wav2vec2-large-xlsr-53-hebrew")
model = Wav2Vec2ForCTC.from_pretrained("imvladikon/wav2vec2-large-xlsr-53-hebrew").to("cuda")
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'
resampler = torchaudio.transforms.Resample(48_000, 16_000)
# Preprocessing the datasets.
# We need to read the aduio files as arrays
def speech_file_to_array_fn(batch):
  batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower()
  speech_array, sampling_rate = torchaudio.load(batch["path"])
  batch["speech"] = resampler(speech_array).squeeze().numpy()
  return batch
test_dataset = test_dataset.map(speech_file_to_array_fn)
# Preprocessing the datasets.
# We need to read the aduio files as arrays
def evaluate(batch):
  inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)
  with torch.no_grad():
    logits = model(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits
  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_strings"] = processor.batch_decode(pred_ids)
  return batch
result = test_dataset.map(evaluate, batched=True, batch_size=8)

print("WER: {:2f}".format(100 * wer.compute(predictions=result["pred_strings"], references=result["sentence"])))

In [1]:
import faster_whisper

# Load the Whisper model (change 'ivrit-ai/faster-whisper-v2-d4' to the correct model path if necessary)
model = faster_whisper.WhisperModel('ivrit-ai/faster-whisper-v2-d4')

# Specify the path to the media file (audio or video)
media_file_path = '/home/ameer/Kaleidoo/Data/Audio_Data/Hebrow/test.mp3'  # Replace with actual media file

# Transcribe the media file with the model, specifying the language
segs, _ = model.transcribe(media_file_path, language='he')  # 'he' for Hebrew

# Extract the text segments from the transcription
texts = [s.text for s in segs]

# Join all the text segments into a single string
transcribed_text = ' '.join(texts)

# Print the final transcribed text
print(f'Transcribed text: {transcribed_text}')


/home/ameer/Kaleidoo/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2024-09-19 09:58:08.920] [ctranslate2] [thread 143738] [warning] The compute type inferred from the saved model is float16, but the target device or backend do not support efficient float16 computation. The model weights have been automatically converted to use the float32 compute type instead.
